# LUIS Model

In this notebook, we :
- create an Azure LUIS **resource**
- add the **intents** and **entities**
- **format** the data to be compatible with LUIS
- add the **examples**
- run a LUIS model **training**

In [1]:
## Download and extract dataset files

!cd .. && make dataset && cd notebooks

>>> Downloading and saving data files...
Data files already downloaded.
>>> OK.



In [2]:
## Import and configure libraries

import os
import time
import warnings
from pathlib import Path

import pandas as pd
from azure.cognitiveservices.language.luis.authoring import LUISAuthoringClient
from azure.cognitiveservices.language.luis.authoring.models import (
    ApplicationCreateObject,
    ExampleLabelObject,
    EntityLabelObject,
)
from dotenv import load_dotenv
from msrest.authentication import CognitiveServicesCredentials
from tqdm.notebook import tqdm_notebook as tqdm

warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

load_dotenv(override=True)

AZURE_LUIS_ENDPOINT = os.getenv("AZURE_LUIS_ENDPOINT")
AZURE_LUIS_KEY = os.getenv("AZURE_LUIS_KEY")

AZURE_LUIS_PROJECT_NAME = "ocp10-luis-model"
AZURE_LUIS_PROJECT_VERSION = "0.1"

DATA_PATH = Path("../data")
FRAMES_JSON_PATH = Path(DATA_PATH, "raw/frames.json")

In [3]:
## Create the LUIS Application

# instanciate LUIS Authoring Client
client = LUISAuthoringClient(
    AZURE_LUIS_ENDPOINT, CognitiveServicesCredentials(AZURE_LUIS_KEY)
)

# define app basics
appDefinition = ApplicationCreateObject(
    name=AZURE_LUIS_PROJECT_NAME,
    culture="en-us",
)

# create app
app_id = client.apps.add(appDefinition)

# get app id - necessary for all other changes
print("Created LUIS app with ID {}".format(app_id))

Created LUIS app with ID b8261ace-7626-4544-87a6-ae4f2ffae907


In [5]:
## Add Intents

book_id = client.model.add_intent(
    app_id=app_id, version_id=AZURE_LUIS_PROJECT_VERSION, name="Book"
)
info_id = client.model.add_intent(
    app_id=app_id, version_id=AZURE_LUIS_PROJECT_VERSION, name="Info"
)

In [6]:
## Add entities

# add Prebuilt entity
client.model.add_prebuilt(
    app_id=app_id,
    version_id=AZURE_LUIS_PROJECT_VERSION,
    prebuilt_extractor_names=["geographyV2"],
)
client.model.add_prebuilt(
    app_id=app_id,
    version_id=AZURE_LUIS_PROJECT_VERSION,
    prebuilt_extractor_names=["datetimeV2"],
)
client.model.add_prebuilt(
    app_id=app_id,
    version_id=AZURE_LUIS_PROJECT_VERSION,
    prebuilt_extractor_names=["number"],
)

# add entity to app
or_city_id = client.model.add_entity(
    app_id=app_id, version_id=AZURE_LUIS_PROJECT_VERSION, name="or_city"
)
client.features.add_entity_feature(
    app_id=app_id,
    version_id=AZURE_LUIS_PROJECT_VERSION,
    entity_id=or_city_id,
    feature_relation_create_object={
        "model_name": "geographyV2",
    },
)
dst_city_id = client.model.add_entity(
    app_id=app_id, version_id=AZURE_LUIS_PROJECT_VERSION, name="dst_city"
)
client.features.add_entity_feature(
    app_id=app_id,
    version_id=AZURE_LUIS_PROJECT_VERSION,
    entity_id=dst_city_id,
    feature_relation_create_object={
        "model_name": "geographyV2",
    },
)
str_date_id = client.model.add_entity(
    app_id=app_id, version_id=AZURE_LUIS_PROJECT_VERSION, name="str_date"
)
client.features.add_entity_feature(
    app_id=app_id,
    version_id=AZURE_LUIS_PROJECT_VERSION,
    entity_id=str_date_id,
    feature_relation_create_object={
        "model_name": "datetimeV2",
    },
)
end_date_id = client.model.add_entity(
    app_id=app_id, version_id=AZURE_LUIS_PROJECT_VERSION, name="end_date"
)
client.features.add_entity_feature(
    app_id=app_id,
    version_id=AZURE_LUIS_PROJECT_VERSION,
    entity_id=end_date_id,
    feature_relation_create_object={
        "model_name": "datetimeV2",
    },
)
budget_id = client.model.add_entity(
    app_id=app_id, version_id=AZURE_LUIS_PROJECT_VERSION, name="budget"
)
client.features.add_entity_feature(
    app_id=app_id,
    version_id=AZURE_LUIS_PROJECT_VERSION,
    entity_id=budget_id,
    feature_relation_create_object={
        "model_name": "number",
    },
)

In [7]:
## Format data for LUIS

raw_data = pd.read_json(FRAMES_JSON_PATH)


entities = ["or_city", "dst_city", "str_date", "end_date", "budget"]
examples = []
unique_utterances = []

for turn in tqdm(raw_data["turns"]):
    for frame in turn:
        if frame["author"] == "wizard" or frame["text"] in unique_utterances:
            continue

        unique_utterances.append(frame["text"])

        is_book = False
        labels = []

        for act in frame["labels"]["acts_without_refs"]:
            for arg in act["args"]:
                if arg["key"] == "intent" and arg["val"] == "book":
                    is_book = True

                if (
                    arg["key"] in entities
                    and arg["val"] is not None
                    and frame["text"].find(arg["val"]) != -1
                ):
                    labels.append(
                        EntityLabelObject(
                            entity_name=arg["key"],
                            start_char_index=frame["text"].find(arg["val"]),
                            end_char_index=frame["text"].find(arg["val"])
                            + len(arg["val"]),
                        )
                    )

        if len(entities) > 0:
            examples.append(
                ExampleLabelObject(
                    text=frame["text"],
                    intent_name="Book" if is_book else "Info",
                    entity_labels=labels,
                )
            )

# add the examples in batch
batch_size = 100
for index in tqdm(range(0, len(examples), batch_size)):
    client.examples.batch(
        app_id=app_id,
        version_id=AZURE_LUIS_PROJECT_VERSION,
        example_label_object_array=examples[index : index + batch_size],
    )

  0%|          | 0/1369 [00:00<?, ?it/s]

  0%|          | 0/97 [00:00<?, ?it/s]

In [8]:
## Train the model to recognise intents and entities in examples

client.train.train_version(app_id=app_id, version_id=AZURE_LUIS_PROJECT_VERSION)
waiting = True
while waiting:
    info = client.train.get_status(app_id=app_id, version_id=AZURE_LUIS_PROJECT_VERSION)

    # get_status returns a list of training statuses, one for each model. Loop through them and make sure all are done.
    waiting = any(
        map(
            lambda x: "Queued" == x.details.status or "InProgress" == x.details.status,
            info,
        )
    )
    if waiting:
        print("Waiting 10 seconds for training to complete...")
        time.sleep(10)
    else:
        print("trained")
        waiting = False

Waiting 10 seconds for training to complete...
Waiting 10 seconds for training to complete...
Waiting 10 seconds for training to complete...
Waiting 10 seconds for training to complete...
Waiting 10 seconds for training to complete...
Waiting 10 seconds for training to complete...
Waiting 10 seconds for training to complete...
Waiting 10 seconds for training to complete...
Waiting 10 seconds for training to complete...
Waiting 10 seconds for training to complete...
Waiting 10 seconds for training to complete...
Waiting 10 seconds for training to complete...
Waiting 10 seconds for training to complete...
Waiting 10 seconds for training to complete...
Waiting 10 seconds for training to complete...
Waiting 10 seconds for training to complete...
Waiting 10 seconds for training to complete...
Waiting 10 seconds for training to complete...
Waiting 10 seconds for training to complete...
Waiting 10 seconds for training to complete...
Waiting 10 seconds for training to complete...
Waiting 10 se

In [15]:
## Test the model

import requests
import json


AZURE_LUIS_PRED_ENDPOINT = os.getenv("AZURE_LUIS_PRED_ENDPOINT")
AZURE_LUIS_PRED_KEY = os.getenv("AZURE_LUIS_PRED_KEY")

query = "I want to book a trip from Paris to London for less than $100. I will leave on the first of January 2023 and come back on the 17th of january 2023."

pred = requests.get(
    f"{AZURE_LUIS_PRED_ENDPOINT}?verbose=true&show-all-intents=true&log=true&subscription-key={AZURE_LUIS_PRED_KEY}&query={query}"
).json()


print(json.dumps(pred, indent=4))

{
    "query": "I want to book a trip from Paris to London for less than $100. I will leave on the first of January 2023 and come back on the 17th of january 2023.",
    "prediction": {
        "topIntent": "Book",
        "intents": {
            "Book": {
                "score": 0.9316537
            },
            "Info": {
                "score": 0.2502006
            },
            "None": {
                "score": 0.0001811284
            }
        },
        "entities": {
            "or_city": [
                "Paris"
            ],
            "geographyV2": [
                {
                    "value": "Paris",
                    "type": "city"
                },
                {
                    "value": "London",
                    "type": "city"
                }
            ],
            "dst_city": [
                "London"
            ],
            "budget": [
                "$100."
            ],
            "number": [
                100,
           